In [ ]:
# pip install snowflake-snowpark-python boto3 

In [ ]:
import boto3
import io
import os
import csv
import datetime
from snowflake.snowpark import Session

In [ ]:
aws_access_key = 'XXXXXXXXXXXXXXXX'
aws_secret_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
aws_s3_bucket_name = 'praveeng-s3'  
aws_s3_object = 'chunked_csv/'
s3 = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)
chunked_csv_folder = r"C:\Users\DELL\Desktop\aws_to_sf\s3_chunked_csv"

connection_parameters = {
   "account": "dr31778.ca-central-1.aws",
   "user": "PRAVEEN11001",
   "password": "XXXXXXXXXXXXXXXX",
   "role": "ACCOUNTADMIN",  # optional
   "warehouse": "COMPUTE_WH",  # optional
   "database": "DEMO",  # optional
   "schema": "s3"  # optional
}

session = Session.builder.configs(connection_parameters).create()

In [ ]:
def delete_chunked_files():
    try:
        for file in os.listdir(chunked_csv_folder):
            file_path = os.path.join(chunked_csv_folder, file)
            os.remove(file_path)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
def load_into_fs_stage(file_to_load):
    put_result = session.sql(f"PUT file://{file_to_load} @fs_stage;").collect()
    print(put_result)

In [ ]:
def copy_into_sf(session, filename):
    copy_date = datetime.datetime.utcnow()
    
    copy_result = session.sql(f"""
    COPY INTO AWS_STUDENT_DATASET_FS(filename, ID, NAME, NATIONALITY, CITY, 
                              LATITUDE, LONGITUDE, GENDER, ethinic_group, 
                              AGE, ENGLISH_GRADE, MATH_GRADE, SCIENCE_GRADE, 
                              LANGUAGE_GRADE, PORTFOLIO_RATING, COVERLETTER_RATING, 
                              REFLETTER_RATING, copy_date)
    FROM 
        (SELECT '{filename}' AS filename, 
            $1 AS ID, 
            $2 AS NAME, 
            $3 AS NATIONALITY, 
            $4 AS CITY, 
            $5 AS LATITUDE, 
            $6 AS LONGITUDE,
            $7 AS GENDER, 
            $8 AS ethinic_group, 
            $9 AS AGE, 
            $10 AS ENGLISH_GRADE, 
            $11 AS MATH_GRADE, 
            $12 AS SCIENCE_GRADE, 
            $13 AS LANGUAGE_GRADE, 
            $14 AS PORTFOLIO_RATING, 
            $15 AS COVERLETTER_RATING, 
            $16 AS REFLETTER_RATING,
            '{copy_date}' AS copy_date 
        FROM @fs_stage/{filename})
        
    ON_ERROR='SKIP_FILE'
    FILE_FORMAT= (FORMAT_NAME=CSV_FORMAT);
    """).collect()

    print(copy_result)

In [ ]:
def load_csv_sf(local_csv_directory):
    for file_name in os.listdir(local_csv_directory):
        if file_name.endswith('.csv'):
            full_path = os.path.join(local_csv_directory, file_name)

            load_into_fs_stage(full_path)
            copy_into_sf(session, file_name)            

In [ ]:
response = s3.list_objects_v2(Bucket=aws_s3_bucket_name, Prefix='xmls')

for obj in response['Contents']:
    key = obj['Key']
    if key.endswith('set.csv'):
        obj_size = obj['Size']
        response = s3.get_object(Bucket=aws_s3_bucket_name, Key=key)
        file_content = response['Body'].read().decode('utf-8')

        if obj_size > 10 * 1024:
            chunk_size = 10 * 1024  
            reader = csv.reader(io.StringIO(file_content))
            headers = next(reader)  
            chunks = []
            current_chunk = [','.join(headers)]  # Include header in the first chunk
            current_chunk_size = len(','.join(headers).encode('utf-8'))

            for row in reader:
                row_str = ','.join(row)
                row_size = len(row_str.encode('utf-8'))
                if current_chunk_size + row_size <= chunk_size:
                    current_chunk.append(row_str)
                    current_chunk_size += row_size
                else:
                    chunks.append(current_chunk)
                    current_chunk = [','.join(headers), row_str]  
                    current_chunk_size = len(','.join(headers).encode('utf-8')) + row_size

            if current_chunk:
                chunks.append(current_chunk)

            for i, chunk in enumerate(chunks):
                chunked_filename = f"{chunked_csv_folder}\{key.split('/')[1].split('.')[0]}_{i}.csv".replace('-', '_')
                partition_data = '\n'.join(chunk)
                with open(chunked_filename, 'w', encoding='utf-8') as file:
                    file.write(partition_data)
                print(f"saved partition {i + 1} of {key} to local file system: {chunked_filename}")
        else:
            print(f"saved partition {i + 1} of {key} to local file system: {chunked_filename}")
            chunked_filename = f"{chunked_csv_folder}{key.split('/')[1]}".replace('-', '_')
            with open(chunked_filename, 'w', encoding='utf-8') as file:
                file.write(partition_data, )

In [ ]:
load_csv_sf(chunked_csv_folder)

In [ ]:
delete_chunked_files()